# **Modelado con XGBoost**

## **1. Importación de librerías**

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from xgboost import XGBClassifier
import optuna
import pickle
import warnings
warnings.filterwarnings('ignore')

# Configurar seeds para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## **2. Cargar Datos y Verificar Balance de Clases**

In [115]:
## 2. Cargar y Analizar los Datos Preprocesados

# Cargar el dataset preprocesado
df = pd.read_csv('../../data/processed/preprocessing.csv')


print(f"Distribución de clases:")
for i, (class_val, count) in enumerate(stroke_counts.items()):
    percentage = stroke_percentages[class_val]
    print(f"  Clase {class_val}: {count:,} casos ({percentage:.2f}%)")

# Calcular el ratio de desbalance
minority_class = stroke_counts.min()
majority_class = stroke_counts.max()
imbalance_ratio = majority_class / minority_class

print(f"\n📈 Ratio de desbalance: {imbalance_ratio:.2f}:1")
print(f"💡 Interpretación: La clase mayoritaria es {imbalance_ratio:.1f} veces más grande que la minoritaria")

if imbalance_ratio > 5:
    print("⚠️  DATASET MUY DESBALANCEADO - Se recomienda usar SMOTE")
elif imbalance_ratio > 2:
    print("⚠️  DATASET MODERADAMENTE DESBALANCEADO - Considerar técnicas de balanceo")
else:
    print("✅ DATASET RELATIVAMENTE BALANCEADO")

Distribución de clases:
  Clase 0: 4,733 casos (95.02%)
  Clase 1: 248 casos (4.98%)

📈 Ratio de desbalance: 19.08:1
💡 Interpretación: La clase mayoritaria es 19.1 veces más grande que la minoritaria
⚠️  DATASET MUY DESBALANCEADO - Se recomienda usar SMOTE


## **3.  División de Datos y Aplicación de SMOTE**

In [116]:
## 3. División de Datos y Configuración de Class Weights

# Separar características (X) y variable objetivo (y)
X = df.drop('stroke', axis=1)
y = df['stroke']

print(f"📊 Características (X): {X.shape}")
print(f"🎯 Variable objetivo (y): {y.shape}")

# División inicial en train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y  # Mantener proporción de clases
)

print(f"\n📈 DIVISIÓN DE DATOS:")
print(f"  Train: {X_train.shape[0]} casos")
print(f"  Test: {X_test.shape[0]} casos")

# Analizar distribución para calcular class weights
print(f"\n🎯 DISTRIBUCIÓN EN TRAIN:")
train_counts = y_train.value_counts()
for class_val, count in train_counts.items():
    percentage = (count / len(y_train)) * 100
    print(f"  Clase {class_val}: {count:,} casos ({percentage:.2f}%)")

# Calcular scale_pos_weight para XGBoost
# Esto le dice al modelo qué tan importante es la clase minoritaria
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

print(f"\n⚖️ CONFIGURACIÓN DE CLASS WEIGHTS:")
print(f"  scale_pos_weight = {scale_pos_weight:.2f}")
print(f"  📝 Significado: Los errores en clase 1 (stroke) pesan {scale_pos_weight:.1f}x más")
print(f"  💡 El modelo será {scale_pos_weight:.1f}x más cuidadoso detectando strokes")

# Verificar distribución en test (debe mantenerse similar)
print(f"\n🧪 DISTRIBUCIÓN EN TEST (verificación):")
test_counts = y_test.value_counts()
for class_val, count in test_counts.items():
    percentage = (count / len(y_test)) * 100
    print(f"  Clase {class_val}: {count:,} casos ({percentage:.2f}%)")

print(f"\n✅ DATOS LISTOS PARA ENTRENAMIENTO")
print(f"   - Usaremos los {len(y_train):,} casos reales de train")
print(f"   - XGBoost aplicará class weights automáticamente")
print(f"   - NO hay datos sintéticos (más realista)")

📊 Características (X): (4981, 14)
🎯 Variable objetivo (y): (4981,)

📈 DIVISIÓN DE DATOS:
  Train: 3984 casos
  Test: 997 casos

🎯 DISTRIBUCIÓN EN TRAIN:
  Clase 0: 3,786 casos (95.03%)
  Clase 1: 198 casos (4.97%)

⚖️ CONFIGURACIÓN DE CLASS WEIGHTS:
  scale_pos_weight = 19.12
  📝 Significado: Los errores en clase 1 (stroke) pesan 19.1x más
  💡 El modelo será 19.1x más cuidadoso detectando strokes

🧪 DISTRIBUCIÓN EN TEST (verificación):
  Clase 0: 947 casos (94.98%)
  Clase 1: 50 casos (5.02%)

✅ DATOS LISTOS PARA ENTRENAMIENTO
   - Usaremos los 3,984 casos reales de train
   - XGBoost aplicará class weights automáticamente
   - NO hay datos sintéticos (más realista)


In [117]:
## 4. Modelo Base XGBoost y Evaluación Completa

# Configurar validación cruzada
cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Crear modelo base con class weights (SIN early stopping por ahora)
xgb_base = xgb.XGBClassifier(
    scale_pos_weight=scale_pos_weight,  # 19.12 para balancear clases
    random_state=RANDOM_STATE,
    eval_metric='logloss',
    n_estimators=100,  # Número fijo de árboles para modelo base
    verbose=False
)

print("🚀 ENTRENANDO MODELO BASE XGBOOST")
print("=" * 50)

# Entrenar modelo base
xgb_base.fit(X_train, y_train)

# Predicciones
y_train_pred = xgb_base.predict(X_train)
y_train_proba = xgb_base.predict_proba(X_train)[:, 1]
y_test_pred = xgb_base.predict(X_test)
y_test_proba = xgb_base.predict_proba(X_test)[:, 1]

print("✅ Modelo entrenado correctamente")

# MÉTRICAS EN TRAIN
print(f"\n📊 MÉTRICAS EN TRAIN:")
print("=" * 30)
train_accuracy = (y_train_pred == y_train).mean()
train_f1 = f1_score(y_train, y_train_pred)
train_auc = roc_auc_score(y_train, y_train_proba)

print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1-Score: {train_f1:.4f}")
print(f"AUC-ROC:  {train_auc:.4f}")

print(f"\nClassification Report TRAIN:")
print(classification_report(y_train, y_train_pred))

# MÉTRICAS EN TEST
print(f"\n📊 MÉTRICAS EN TEST:")
print("=" * 30)
test_accuracy = (y_test_pred == y_test).mean()
test_f1 = f1_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_proba)

print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1-Score: {test_f1:.4f}")
print(f"AUC-ROC:  {test_auc:.4f}")

print(f"\nClassification Report TEST:")
print(classification_report(y_test, y_test_pred))

# ANÁLISIS DE OVERFITTING
print(f"\n🔍 ANÁLISIS DE OVERFITTING:")
print("=" * 40)
accuracy_diff = abs(train_accuracy - test_accuracy)
f1_diff = abs(train_f1 - test_f1)
auc_diff = abs(train_auc - test_auc)

print(f"Diferencia Accuracy: {accuracy_diff:.4f} ({accuracy_diff*100:.2f}%)")
print(f"Diferencia F1-Score: {f1_diff:.4f} ({f1_diff*100:.2f}%)")
print(f"Diferencia AUC-ROC:  {auc_diff:.4f} ({auc_diff*100:.2f}%)")

# Criterio de overfitting según el issue: <5%
if accuracy_diff < 0.05:
    print(f"✅ Accuracy: NO overfitting (diferencia < 5%)")
else:
    print(f"⚠️  Accuracy: POSIBLE overfitting (diferencia ≥ 5%)")

if f1_diff < 0.05:
    print(f"✅ F1-Score: NO overfitting (diferencia < 5%)")
else:
    print(f"⚠️  F1-Score: POSIBLE overfitting (diferencia ≥ 5%)")

if auc_diff < 0.05:
    print(f"✅ AUC-ROC: NO overfitting (diferencia < 5%)")
else:
    print(f"⚠️  AUC-ROC: POSIBLE overfitting (diferencia ≥ 5%)")

# VALIDACIÓN CRUZADA
print(f"\n🔄 VALIDACIÓN CRUZADA (5-FOLD):")
print("=" * 40)

# CV con F1
cv_f1_scores = cross_val_score(xgb_base, X_train, y_train, cv=cv_folds, scoring='f1')
print(f"F1-Score CV: {cv_f1_scores.mean():.4f} ± {cv_f1_scores.std():.4f}")
print(f"F1 Scores por fold: {[f'{score:.4f}' for score in cv_f1_scores]}")

# CV con AUC-ROC
cv_auc_scores = cross_val_score(xgb_base, X_train, y_train, cv=cv_folds, scoring='roc_auc')
print(f"AUC-ROC CV: {cv_auc_scores.mean():.4f} ± {cv_auc_scores.std():.4f}")
print(f"AUC Scores por fold: {[f'{score:.4f}' for score in cv_auc_scores]}")

# MATRIZ DE CONFUSIÓN
print(f"\n📋 MATRIZ DE CONFUSIÓN (TEST):")
print("=" * 35)
cm = confusion_matrix(y_test, y_test_pred)
print(f"Verdaderos Negativos (TN): {cm[0,0]}")
print(f"Falsos Positivos (FP):     {cm[0,1]}")
print(f"Falsos Negativos (FN):     {cm[1,0]}")
print(f"Verdaderos Positivos (TP): {cm[1,1]}")

# Calcular Precision y Recall manualmente para verificar
precision = cm[1,1] / (cm[1,1] + cm[0,1]) if (cm[1,1] + cm[0,1]) > 0 else 0
recall = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1] + cm[1,0]) > 0 else 0

print(f"\nPrecision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")

print(f"\n🎯 RESUMEN MODELO BASE:")
print("=" * 30)
print(f"✅ F1-Score Test: {test_f1:.4f}")
print(f"✅ AUC-ROC Test:  {test_auc:.4f}")
print(f"✅ CV F1 promedio: {cv_f1_scores.mean():.4f}")
print(f"✅ Diferencia Train/Test: {max(accuracy_diff, f1_diff, auc_diff)*100:.2f}%")

🚀 ENTRENANDO MODELO BASE XGBOOST
✅ Modelo entrenado correctamente

📊 MÉTRICAS EN TRAIN:
Accuracy: 0.9990
F1-Score: 0.9900
AUC-ROC:  1.0000

Classification Report TRAIN:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3786
           1       0.98      1.00      0.99       198

    accuracy                           1.00      3984
   macro avg       0.99      1.00      0.99      3984
weighted avg       1.00      1.00      1.00      3984


📊 MÉTRICAS EN TEST:
Accuracy: 0.9258
F1-Score: 0.1395
AUC-ROC:  0.8023

Classification Report TEST:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       947
           1       0.17      0.12      0.14        50

    accuracy                           0.93       997
   macro avg       0.56      0.54      0.55       997
weighted avg       0.91      0.93      0.92       997


🔍 ANÁLISIS DE OVERFITTING:
Diferencia Accuracy: 0.0732 (7.32%)
Diferencia F

AttributeError: 'super' object has no attribute '__sklearn_tags__'